In [14]:
import pandas as pd

district_data = pd.read_csv('cleaned_district_data.csv')
county_data = pd.read_csv('county_demographics_coding_task.csv')
county_data.head()



,congress,year,state,county,district,cd_share_of_county,single_district_county,population,white,black,...,age55_59,age5_9,age60_64,age65_69,age70_74,age75_79,age80_84,age85over,treatment,treatment_change
0,117,2022,1,1001,2,1.0,1,58805,42160,11445,...,4147,3796,3624,3006,2414,1798,1098,927,0.406292,0.006191
1,117,2021,1,1001,2,1.0,1,58805,42160,11445,...,4147,3796,3624,3006,2414,1798,1098,927,0.406292,0.006191
2,116,2020,1,1001,2,1.0,1,58805,42160,11445,...,4147,3796,3624,3006,2414,1798,1098,927,0.406292,0.006191
3,116,2019,1,1001,2,1.0,1,54571,42855,9643,...,3083,3991,2777,2277,1736,1251,731,551,0.406292,0.006191
4,115,2018,1,1001,2,1.0,1,54571,42855,9643,...,3083,3991,2777,2277,1736,1251,731,551,0.406292,0.006191


In [15]:
# merging county and district datasets



# convert FIPS codes to state abbreviations
fips_to_abbrev = {1:'AL', 2:'AK', 4:'AZ', 6:'CA', 8:'CO', 9:'CT', 10:'DE', 11:'DC', 
                  12:'FL', 13:'GA', 15:'HI', 16:'ID', 17:'IL', 18:'IN', 19:'IA', 20:'KS',
                  21:'KY', 22:'LA', 23:'ME', 24:'MD', 25:'MA', 26:'MI', 27:'MN', 28:'MS',
                  29:'MO', 30:'MT', 31:'NE', 32:'NV', 33:'NH', 34:'NJ', 35:'NM', 36:'NY',
                  37:'NC', 38:'ND', 39:'OH', 40:'OK', 41:'OR', 42:'PA', 72:'PR', 44:'RI',
                  45:'SC', 46:'SD', 47:'TN', 48:'TX', 49:'UT', 50:'VT', 51:'VA', 78:'VI',
                  53:'WA', 54:'WV', 55:'WI', 56:'WY'}

county_data['state'] = county_data['state'].map(fips_to_abbrev)

# converts districts listed with state and number to number (i.e. TX01 --> 1)
district_data['district'] = district_data['district'].str.extract('(\d+)').astype(int)

# drops rows with odd years
county_data = county_data.drop( county_data[county_data['year'] % 2 != 0].index, axis=0)

# merges data based on state, district, and year
county_data = county_data.merge(district_data, on = ['state', 'district','year'], how = 'left')
#print(county_data)
# confirms that only the state category has NaN values 
county_data['state'].isnull().unique() # = [False, True]
county_data['district'].isnull().unique() # = [False]
county_data['year'].isnull().unique() # [False]

# drops state/district/year row if either state, district, or year contains an NaN value

NaN_removed = county_data[['state', 'district', 'year']].dropna() 

# drops observations from district that did not match county by ensuring that the state/district/year combinations of district_data match NaN_removed 
district_data = district_data[ district_data[['state', 'district', 'year']].apply(tuple, axis=1).isin(NaN_removed)]


# identifies where county data did not match district data (any NaN entry) and applies single incumbent label
NaN_rows = county_data[ county_data[['state', 'district', 'year']].isnull().any(axis=1)]

# applies 0,1,2.. indexing to county_data to easily identify the NaN rows
county_data = county_data.reset_index(drop = True)


county_data.loc[NaN_rows.index, 'one_incumb'] = True
county_data.loc[NaN_rows.index, 'zero_incumbs'] = False
county_data.loc[NaN_rows.index, 'multiple_incumbs'] = False


county_data.to_csv('/Users/divya/Documents/ra_data_task/data/merged_county_district_data.csv')


county_data


                  

<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
/var/folders/11/1zlp690171z58pnrzbnr1s380000gn/T/ipykernel_9413/2265944055.py:17: SyntaxWarning: invalid escape sequence '\d'
  district_data['district'] = district_data['district'].str.extract('(\d+)').astype(int)


[True False nan]
[False True nan]
[False True nan]


,congress,year,state,county,district,cd_share_of_county,single_district_county,population,white,black,...,age75_79,age80_84,age85over,treatment,treatment_change,Unnamed: 0,ico.status,zero_incumbs,one_incumb,multiple_incumbs
0,117,2022,AL,1001,2,1.0,1,58805,42160,11445,...,1798,1098,927,0.406292,0.006191,39.0,C,True,False,False
1,117,2022,AL,1001,2,1.0,1,58805,42160,11445,...,1798,1098,927,0.406292,0.006191,40.0,I,False,True,False
2,116,2020,AL,1001,2,1.0,1,58805,42160,11445,...,1798,1098,927,0.406292,0.006191,36.0,C,True,False,False
3,116,2020,AL,1001,2,1.0,1,58805,42160,11445,...,1798,1098,927,0.406292,0.006191,37.0,I,False,True,False
4,116,2020,AL,1001,2,1.0,1,58805,42160,11445,...,1798,1098,927,0.406292,0.006191,38.0,O,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62031,109,2006,WY,56045,1,1.0,1,6644,6374,8,...,217,157,134,0.497579,0.000000,8685.0,I,False,True,False
62032,108,2004,WY,56045,1,1.0,1,6644,6374,8,...,217,157,134,0.497579,0.000000,8682.0,C,True,False,False
62033,108,2004,WY,56045,1,1.0,1,6644,6374,8,...,217,157,134,0.497579,0.000000,8683.0,I,False,True,False
62034,107,2002,WY,56045,1,1.0,1,6644,6374,8,...,217,157,134,0.497579,0.000000,8680.0,C,True,False,False
